In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import optimizers

df = pd.read_csv('dicindia.csv')

Using TensorFlow backend.


In [2]:
df.head(10)

Date    Open    High     Low   Close  Volume
0  01/07/20  327.90  341.00  326.95  334.00     871
1  30/06/20  349.00  349.00  329.80  334.70    3162
2  29/06/20  343.30  343.30  336.00  336.00     160
3  26/06/20  354.90  354.90  340.00  343.30     127
4  25/06/20  335.20  340.10  335.20  340.10     119
5  24/06/20  344.80  348.05  344.00  345.20     516
6  23/06/20  350.15  354.00  346.05  346.05     933
7  22/06/20  325.00  356.15  325.00  350.35    2068
8  19/06/20  349.00  360.30  341.30  348.10    5747
9  18/06/20  308.00  310.00  308.00  310.00      83

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
df.loc[:,'Close'] = scaler.fit_transform(np.array(df.loc[:,'Close']).reshape(-1,1))

In [4]:
def split_sequence(sequence, n_steps):
    X = []
    y = []
    for i in range(len(sequence)):
        end_idx = i + n_steps
        if end_idx > len(sequence) - 1:
            break
        Xtemp = sequence[i:end_idx]
        ytemp = sequence[end_idx]
        X.append(Xtemp)
        y.append(ytemp)
    
    return np.array(X), np.array(y)

In [5]:
train_len = int(len(df.loc[:,'Close']) * 0.7)
test_len = int(len(df.loc[:,'Close']) * 0.3)

train_data = df.loc[:train_len,'Close']
test_data = df.loc[train_len:,'Close']

In [6]:
timesteps = 7

X_train, y_train = split_sequence(np.array(train_data), timesteps)
X_test, y_test = split_sequence(np.array(test_data), timesteps)

In [7]:
n_features = 1
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], n_features)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], n_features)

model = Sequential()
model.add(LSTM(32, activation = 'relu', return_sequences=True, input_shape=(timesteps,n_features)))
# model.add(LSTM(32, activation = 'relu', return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(optimizer='adam', loss="mean_squared_error", metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 7, 32)             4352      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 12,705
Trainable params: 12,705
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(X_train, y_train, epochs = 100, batch_size=30)

Epoch 1/100
691/691 [==============================] - 4s 6ms/step - loss: 0.0513 - acc: 0.0014
Epoch 2/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0104 - acc: 0.0014
Epoch 3/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0029 - acc: 0.0014
Epoch 4/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0012 - acc: 0.0014
Epoch 5/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0012 - acc: 0.0014
Epoch 6/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0012 - acc: 0.0014
Epoch 7/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0013 - acc: 0.0014
Epoch 8/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0011 - acc: 0.0014
Epoch 9/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0011 - acc: 0.0014
Epoch 10/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0011 - acc: 0.0014
Epoch 11/100
691/691 [=================

In [13]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

In [14]:
import math
from sklearn.metrics import mean_squared_error

rmse_train = math.sqrt(mean_squared_error(y_train, train_predict))
rmse_test = math.sqrt(mean_squared_error(y_test, test_predict))
print(rmse_train)
print("\n")
print(rmse_test)

427.15126704718904


553.7807482567541
